In [ ]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from glob import glob
from os.path import join, exists, splitext, split
from statistics import mean
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
nx = 9 #number of inside corners in x
ny = 6 #number of inside corners in y
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

In [ ]:
def configure_calibration(calibration_images_dir):
    '''
    Get main path for images to caliprate camera with as input then save a pickle file

    with the configuration to be used in undistortion later
    '''
    #if not exists('calibration_conf.p'):
    # For every calibration image, get object points and image points by finding chessboard corners.
    objpoints = []  # 3D points in real world space.
    imgpoints = []  # 2D points in image space.

    # Prepare constant object points, like (0,0,0), (1,0,0), (2,0,0) ....,(9,6,0).
    objpoints_const = np.zeros((nx * ny, 3), np.float32)
    objpoints_const[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

    images = glob(join(calibration_images_dir, '*.jpg'))
    for img_path in images:
        img = cv2.imread(img_path)
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)   
        if ret == True:
            # append found corners to imgpoints & prepared constants object points to objpoints for mapping
            objpoints.append(objpoints_const)
            imgpoints.append(corners)

    #use all point got from images for calibration
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    # Save to pickle file
    pickle.dump({'mtx': mtx, 'dist': dist, 'corners': corners}, open('calibration_conf.p', 'wb'))


def undistort_image(img, calibration_images_dir = './camera_cal/'):
    '''
    Get calibration configuration from config file to undistort images

    Takes image as input then 
    Returns undistorted image

    '''
    if not exists('calibration_conf.p'):
        configure_calibration(calibration_images_dir)

    # Return pickled calibration data.
    pickle_dict = pickle.load(open('calibration_conf.p', 'rb'))
    mtx = pickle_dict['mtx']
    dist = pickle_dict['dist']
    corners = pickle_dict['corners']

    # return undistorted image
    undist = cv2.undistort(img, mtx, dist, None, mtx)

    return undist, corners

def abs_sobel_thresh(img, sobel_thresh=(0, 255)):
    
    # Apply the following steps to img
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Take the derivative in x & take the absolute value of the result
    abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
        
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)

    # Return this mask as your binary_output image
    binary_output[(scaled_sobel > sobel_thresh[0]) & (scaled_sobel < sobel_thresh[1])] = 1

    return binary_output

def dir_threshold(img, sobel_kernel=3, dir_thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1])] = 1

    # Return the binary image
    return binary_output

def hls_select(img, s_thresh=(0, 255), l_thresh=(0, 255)):
    # Get hls of the image
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # Get the s & l channels to apply thresholds
    s_channel = hls[:,:,2]
    l_channel = hls[:,:,1]
    # Get binary output of s_channel applying thresholds
    binary_output_s = np.zeros_like(s_channel)
    binary_output_s[(s_channel > s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Get binary output of l_channel applying thresholds
    binary_output_l = np.zeros_like(l_channel)
    binary_output_l[(l_channel > l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    
    # Combine the thresholds of both S & L
    combined_binary = np.zeros_like(l_channel)
    combined_binary[(binary_output_l == 1) & (binary_output_s == 1)] = 1
    
    return combined_binary


def birdeye(undist, corners, inverse = False):
    '''
    Get undistort image, corners got from findChessboardCorners as input
    Returns birdeye image for this image
    '''

    # Grab the image shape
    img_size = (undist.shape[1], undist.shape[0])

    # The lower points should be as close to the lower edge of the image as possible.
    # The length of the road in the selected area should be around 30m.
    src = np.float32([(250, 680), (1050, 680), (600, 470), (730, 470)])

    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough to make lines appear parraled
    dst = np.float32([(280, 720), (1000, 720), (280, 0), (1000, 0)])
    # Given src and dst points, calculate the perspective transform matrix
    if inverse:
        M = cv2.getPerspectiveTransform(dst, src)
    else:
        M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(undist, M, img_size)

    return warped



def combined_thresholds(img, sobel_kernel = 3, sobel_thresh=(0, 255), dir_thresh=(0, np.pi/2), s_thresh=(0, 255), l_thresh=(0, 255)):
    '''
    Takes warped image with thresholds as input,
    Calculates the drivative in x direction then returns the result
    '''
    # Convert to HLS color space and separate the S channel
    # Note: img is the undistorted image
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]

    # Sobel filtering in x direction with thresholds to get better view of the edges
    sobel_binary = abs_sobel_thresh(img, sobel_thresh)

    # Get the direction thresholds between 40 to 90 degrees as lane lines are nearly vertical
    dir_binary = dir_threshold(img, sobel_kernel, dir_thresh)
    
    # Get the S & L saturation of the image thersholded for better color saturation & to remove shadows
    sl_binary = hls_select(img, s_thresh, l_thresh)
    
    # Combine the two binary thresholds 
    combined_binary = np.zeros_like(sobel_binary)
    combined_binary[(sobel_binary == 1) & (dir_binary == 1) | (sl_binary == 1) ] = 1

    return combined_binary

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)


    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    
    # Fit a second order polynomial to each using `np.polyfit` real world data
    left_fit_cr = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2)
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    

    return out_img, left_fitx, right_fitx, ploty, left_fit_cr, right_fit_cr

def measure_curvature_pixels(left_fit_cr, right_fit_cr, ploty):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

In [ ]:
def process_image(img):
    undistort, corners = undistort_image(img, calibration_images_dir = './camera_cal/')

    thresholded = combined_thresholds(img, sobel_kernel = 7, sobel_thresh=(10, 100), dir_thresh=(0.69, 1.3), s_thresh=(90, 255), l_thresh=(140, 255))

    #thresholded = hls_select(undistort, s_thresh=(90, 255), l_thresh=(140, 255))
    warped = birdeye(thresholded, corners)

    out_img, left_fitx, right_fitx, ploty, left_fit_cr, right_fit_cr = fit_polynomial(warped)

    left_curverad, right_curverad = measure_curvature_pixels(left_fit_cr, right_fit_cr, ploty)

    #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    #f.tight_layout()
    #ax1.imshow(undistort)
    #ax1.set_title('Original Image', fontsize=50)
    #ax2.imshow(warped)
    #ax2.set_title('Thresholded Grad. Dir.', fontsize=50)
    #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    #plt.imshow(warped)
    #print(left_curverad, right_curverad)
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space
    newwarp = birdeye(color_warp, corners, inverse = True) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistort, 1, newwarp, 0.3, 0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    deviation = (mean(left_fitx) + mean(right_fitx))//2 - 650
    cv2.putText(result,"Left Carvature: "+ str(left_curverad)+"  Right Carvature: "+ str(right_curverad),(100,100), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result,"Deviation from the center: "+ str(deviation * xm_per_pix),(100,150), font, 1,(255,255,255),2,cv2.LINE_AA)

    return result
    


In [ ]:
image_paths = glob(join('./test_images/', '*.jpg'))

for img_path in image_paths:
    img = cv2.imread(img_path)
    result = process_image(img)
    head, tail = split(img_path)
    cv2.imwrite('./output_images/output_' + tail , result)

In [ ]:
output = 'test_videos_output/project_video.mp4'

clip2 = VideoFileClip('project_video.mp4')
clip = clip2.fl_image(process_image)
%time clip.write_videofile(output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

Test Image:

In [ ]:
img_path = './test_images/straight_lines1.jpg'
head, tail = split(img_path)
img = cv2.imread(img_path)

undistort, corners = undistort_image(img, calibration_images_dir = './camera_cal/')
    
cv2.imwrite('./output_images/undistorted_' + tail , undistort)
thresholded = combined_thresholds(img, sobel_kernel = 7, sobel_thresh=(10, 100), dir_thresh=(0.69, 1.3), s_thresh=(90, 255), l_thresh=(140, 255))

cv2.imwrite('./output_images/thresholded_' + tail , thresholded)

#thresholded = hls_select(undistort, s_thresh=(90, 255), l_thresh=(140, 255))
warped = birdeye(thresholded, corners)

cv2.imwrite('./output_images/warped_' + tail , warped)

out_img, left_fitx, right_fitx, ploty, left_fit_cr, right_fit_cr = fit_polynomial(warped)

cv2.imwrite('./output_images/color_fit_lines_' + tail , out_img)

left_curverad, right_curverad = measure_curvature_pixels(left_fit_cr, right_fit_cr, ploty)

#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(undistort)
#ax1.set_title('Original Image', fontsize=50)
#ax2.imshow(warped)
#ax2.set_title('Thresholded Grad. Dir.', fontsize=50)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
#plt.imshow(warped)
#print(left_curverad, right_curverad)
# Create an image to draw the lines on
warp_zero = np.zeros_like(warped).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space
newwarp = birdeye(color_warp, corners, inverse = True) 
# Combine the result with the original image
result = cv2.addWeighted(undistort, 1, newwarp, 0.3, 0)
font = cv2.FONT_HERSHEY_SIMPLEX
deviation = (mean(left_fitx) + mean(right_fitx))//2 - 650
cv2.putText(result,"Left Carvature: "+ str(left_curverad)+"  Right Carvature: "+ str(right_curverad),(100,100), font, 1,(255,255,255),2,cv2.LINE_AA)
cv2.putText(result,"Deviation from the center: "+ str(deviation * xm_per_pix),(100,150), font, 1,(255,255,255),2,cv2.LINE_AA)
cv2.imwrite('./output_images/example_output_' + tail , result)
